In [1]:
import requests
import pandas as pd
import numpy as np
import os

os.chdir('/Users/Alia/Documents/Github/SpendApp/Raw_Data')

# Census/ACS API to Pandas Template

### First, choose your survey of interest. The Census Bureau collects data in many areas through all types surveys, choose the survey you are interested here.
### https://www.census.gov/data/developers/data-sets.html

In [2]:
def get_acs_api_data(base, arr):
    """
    Use the ACS api to get data based on the desired variables.
    Args:
        base (string): base survey url
        arr (list): a list of variables to get the data for (from the ACS api page)
    Returns:
        data (json): the returned data in JSON form
    """
    api_string = ','.join(map(str, arr))
    base_url = f'{base}?get={api_string}';
    api_key = '7db647c83336653be9f2aa68ec8a7a8f5b43154a'; # Alia's personal API key, please don't abuse
    key_extend = f'&key={api_key}'
    url_extend = f'&for=block%20group:*&in=state:01&in=county:*&in=tract:*'; # CHANGE GEOGRAPHY HERE, see API examples page
    url = base_url + url_extend + key_extend
    data = requests.get(url).json();
    return data

def api_data_to_dataframe(data):
    """
    Transform JSON data into Pandas dataframe, setting the GEOID to be the unique geography  identifier.
    Args:
        data (json): the returned data from the ACS api
    Returns:
        df: dataframe with the ACS data
    """
    df = pd.DataFrame(data[1:], columns = data[0]);
    df[f'GEOID'] = df.tract + df.state + df.county; # CHANGE GEOID HERE (e.g. if using county: df[f'GEOID'] = df.state + df.county)
    df.set_index(f'GEOID',inplace=True);
    return df

def clean_data(base, arr):
    """
    Combo method getting the ACS api data and transforming it into a dataframe
    Args:
        base (string): base survey url
        arr (list): a list of variables to get the data for (from the ACS api page)
    Returns:
        df: dataframe with the ACS data 
    """
    api_data = get_acs_api_data(base, arr); # get ACS data from api
    if api_data:
        df = api_data_to_dataframe(api_data); # transform into dataframe, if there is any data
    else:
        df = pd.DataFrame()
    keep_cols = np.append(arr,[f'GEOID']) # remove unneeded columns
    df.drop(columns=[col for col in df if col not in keep_cols], inplace=True)
    return df

def convert_columns_to_int(df,arr):
    """
    Convert the ACS columns to integers, instead of strings
    Args:
        df: ACS data frame
        arr (list): a list of variables to get the data for (from the ACS api page)
    Returns:
        df: ACS data frame
    """
    for i in arr:
        df[i] = df[i].fillna(0)
        df[i] = df[i].astype(int)
    return df

## Next, use API base url to find variables codes
### Variable searches can be done either by variable or by group of variables (browsing by group is easier).

#### Example: 2019 5-year ACS
https://api.census.gov/data/2019/acs/acs5/groups.html

https://api.census.gov/data/2019/acs/acs5/variables.html

As always, be careful of Population vs. Households

In [3]:
over65 = {'B01001_001E':'Total',
        'B01001_020E': 'Male 65-66',
        'B01001_021E': 'Male 67-69',
        'B01001_022E': 'Male 70-74',
        'B01001_023E': 'Male 75-79',
        'B01001_024E': 'Male 80-84',
       'B01001_025E': 'Male >=85',
       'B01001_044E': 'Female 65-66',
        'B01001_045E': 'Female 67-69',
        'B01001_046E': 'Female 70-74',
        'B01001_047E': 'Female 75-79',
        'B01001_048E': 'Female 80-84',
       'B01001_049E': 'Female >=85'}

nohs = {'B06009_001E':'Total',
        'B06009_002E': 'Less than high school graduate'}

nonwhite = {'B01001_001E':'Total',
           'B01001A_001E':'Population White Alone'}

lowinc = {'B06012_001E':'Total',
          'B06012_002E':'Below 100% of poverty level',
          'B06012_003E':'100-149% of poverty level'}

livealone = {'B06012_001E':'Total',
          'B06012_002E':'Below 100% of poverty level',
          'B06012_003E':'100-149% of poverty level'}

In [4]:
def returndf(variables,filename):

    # Dictionary with ACS variable (not group) codes, and the variable name to use in pandas dataframe
    dic = variables

    # Example: 2019 5-year ACS
    base = "https://api.census.gov/data/2019/acs/acs5"

    df = clean_data(base, np.append('NAME', list(dic.keys())))
    df = df.rename(columns=dic)
    df = convert_columns_to_int(df, list(dic.values()))
    
    df.to_csv(f"{filename}.csv",index = False, header=True)
    
    return df

In [5]:
returndf(over65,'Over_65')

,NAME,Total,Male 65-66,Male 67-69,Male 70-74,Male 75-79,Male 80-84,Male >=85,Female 65-66,Female 67-69,Female 70-74,Female 75-79,Female 80-84,Female >=85
GEOID,,,,,,,,,,,,,,
96200001039,"Block Group 2, Census Tract 9620, Covington Co...",884,10,4,4,36,0,0,0,27,15,64,11,0
96180001039,"Block Group 2, Census Tract 9618, Covington Co...",1395,0,16,48,27,11,29,39,5,42,45,58,92
96160001039,"Block Group 4, Census Tract 9616, Covington Co...",793,6,7,46,25,0,0,0,8,37,10,0,18
96160001039,"Block Group 2, Census Tract 9616, Covington Co...",942,17,22,24,7,9,0,18,7,0,33,8,0
96160001039,"Block Group 1, Census Tract 9616, Covington Co...",1726,62,22,27,50,39,23,19,32,50,43,32,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00540601101,"Block Group 2, Census Tract 54.06, Montgomery ...",2072,54,32,19,101,28,0,95,69,34,55,76,18
00540601101,"Block Group 1, Census Tract 54.06, Montgomery ...",427,30,26,17,0,0,0,17,0,17,0,0,0
00560401101,"Block Group 2, Census Tract 56.04, Montgomery ...",642,9,14,8,9,0,11,0,14,8,9,13,0


In [31]:
returndf(nohs,'No_HS_Diploma')

,NAME,Total,Less than high school graduate
GEOID,,,
96200001039,"Block Group 2, Census Tract 9620, Covington Co...",0,0
96180001039,"Block Group 2, Census Tract 9618, Covington Co...",0,0
96160001039,"Block Group 4, Census Tract 9616, Covington Co...",0,0
96160001039,"Block Group 2, Census Tract 9616, Covington Co...",0,0
96160001039,"Block Group 1, Census Tract 9616, Covington Co...",0,0
...,...,...,...
00540601101,"Block Group 2, Census Tract 54.06, Montgomery ...",0,0
00540601101,"Block Group 1, Census Tract 54.06, Montgomery ...",0,0
00560401101,"Block Group 2, Census Tract 56.04, Montgomery ...",0,0


In [35]:
returndf(nonwhite,'Non_white')

,NAME,Total,Population White Alone
GEOID,,,
96200001039,"Block Group 2, Census Tract 9620, Covington Co...",884,0
96180001039,"Block Group 2, Census Tract 9618, Covington Co...",1395,0
96160001039,"Block Group 4, Census Tract 9616, Covington Co...",793,0
96160001039,"Block Group 2, Census Tract 9616, Covington Co...",942,0
96160001039,"Block Group 1, Census Tract 9616, Covington Co...",1726,0
...,...,...,...
00540601101,"Block Group 2, Census Tract 54.06, Montgomery ...",2072,0
00540601101,"Block Group 1, Census Tract 54.06, Montgomery ...",427,0
00560401101,"Block Group 2, Census Tract 56.04, Montgomery ...",642,0


In [19]:
returndf(lowinc,'Low_income')

,NAME,Total,Below 100% of poverty level,100-149% of poverty level
GEOID,,,,
96200001039,"Block Group 2, Census Tract 9620, Covington Co...",0,0,0
96180001039,"Block Group 2, Census Tract 9618, Covington Co...",0,0,0
96160001039,"Block Group 4, Census Tract 9616, Covington Co...",0,0,0
96160001039,"Block Group 2, Census Tract 9616, Covington Co...",0,0,0
96160001039,"Block Group 1, Census Tract 9616, Covington Co...",0,0,0
...,...,...,...,...
00540601101,"Block Group 2, Census Tract 54.06, Montgomery ...",0,0,0
00540601101,"Block Group 1, Census Tract 54.06, Montgomery ...",0,0,0
00560401101,"Block Group 2, Census Tract 56.04, Montgomery ...",0,0,0
